# TRAINING I3W


# A) Create Envorinment, Vehicles etc

### General Parameter

In [1]:
# Define horizon as a variable to ensure consistent use across notebook (length of one rollout)
HORIZON=500

# name of the experiment
experiment_name = "IntersectionExample"

# scenario class
import flow.scenarios as scenarios
print("Available scenarios:")
print(scenarios.__all__)
scenario_name = "IntersectionTWScenario"

# environment class
import flow.envs as flowenvs
print("\nAvailable environments:")
print(flowenvs.__all__)
env_name = "IntersectionEnv"

Available scenarios:
['Scenario', 'BayBridgeScenario', 'BayBridgeTollScenario', 'BottleneckScenario', 'Figure8Scenario', 'SimpleGridScenario', 'HighwayScenario', 'LoopScenario', 'MergeScenario', 'TwoLoopsOneMergingScenario', 'MultiLoopScenario', 'IntersectionScenario', 'IntersectionScenarioTW']

Available environments:
['Env', 'AccelEnv', 'LaneChangeAccelEnv', 'LaneChangeAccelPOEnv', 'LaneChangeAccelEnv_speed', 'GreenWaveTestEnv', 'GreenWaveTestEnv', 'WaveAttenuationMergePOEnv', 'TwoLoopsMergePOEnv', 'BottleneckEnv', 'BottleNeckAccelEnv', 'WaveAttenuationEnv', 'WaveAttenuationPOEnv', 'TrafficLightGridEnv', 'PO_TrafficLightGridEnv', 'DesiredVelocityEnv', 'TestEnv', 'BayBridgeEnv', 'IntersectionEnv']


### Net Parameter

In [2]:
from flow.core.params import NetParams
from flow.scenarios.intersection import ADDITIONAL_NET_PARAMS

additionalNetParams = {
            "edge_length": 40,
            "lanes": 1,
            "speed_limit": 30
        }

net_params = NetParams( no_internal_links=False,                  #default: True   !! damit Kreuzungen nicht überspr. werden
                        inflows=None,                             #default: None
                        osm_path=None,                            #default: None
                        netfile=None,                             #default: None
                        additional_params=additionalNetParams     #default: None   !!
                      )

### InitialConfig Parameter

In [3]:
from flow.core.params import InitialConfig

initial_config = InitialConfig( shuffle=True,                            #default: False         !!
                                spacing="custom",                        #default: "uniform"     !!
                                min_gap=10,                              #default: 0
                                perturbation=30.0,                       #default: 0.0            !!        
                                x0=0,                                    #default: 0
                                bunching=0,                              #default: 0
                                lanes_distribution=float("inf"),         #default: float("inf")
                                edges_distribution="all",                #default: "all"
                                additional_params=None )                 #default: None

### SUMO Parameter

In [4]:
from flow.core.params import SumoParams

sumo_params = SumoParams( port = None,                  #default: None
                          sim_step=0.1,                 #default: 0.1
                          emission_path=None,           #default: None
                          lateral_resolution=None,      #default: None
                          no_step_log=True,             #default: True
                          render=False,                 #default: False
                          save_render=False,            #default: False
                          sight_radius=25,              #default: 25
                          show_radius=False,            #default: False
                          pxpm=2,                       #default: 2
                          overtake_right=False,         #default: False    
                          seed=None,                    #default: None
                          restart_instance=False,       #default: False
                          print_warnings=True,          #default: True
                          teleport_time=-1,             #default: -1
                          num_clients=1,                #default: 1
                          sumo_binary=None )            #default: None

### Environment Parameter

In [5]:
from flow.core.params import EnvParams

additionalEnvParams = {
        # maximum acceleration of autonomous vehicles
        "max_accel": 3,
        # maximum deceleration of autonomous vehicles
        "max_decel": 3,
        "target_velocity": 30
    }

env_params = EnvParams( additional_params=additionalEnvParams, #default: None    !!
                        horizon=HORIZON,                       #default: 500     !!
                        warmup_steps=0,                        #default: 0       
                        sims_per_step=1,                       #default: 1
                        evaluate=False )                       #default: False

### Vehicles Parameter

In [6]:
from flow.core.params import VehicleParams

# import vehicles dynamics models
#from flow.controllers import SumoCarFollowingController
from flow.controllers import ContinuousRouter
#from flow.controllers.lane_change_controllers import SumoLaneChangeController
from flow.controllers.lane_change_controllers import StaticLaneChanger
from flow.controllers import RLController
from flow.core.params import SumoLaneChangeParams
from flow.core.params import SumoCarFollowingParams
from random import *

vehicles = VehicleParams()

#### Add RL-Agent controlled vehicles 

In [7]:
# car following parameters, default: None
cf_parameter = SumoCarFollowingParams(
                speed_mode="aggressive")
# lane change parameters, default: None
lc_parameter =  None

vehicles.add( # name of the vehicle
                veh_id = "rl",
              # acceleration controller, default: (SumoCarFollowingController, {})
                acceleration_controller=(RLController, {}),
              # lane_change_controller, default: (SumoLaneChangeController, {})
                lane_change_controller=(StaticLaneChanger,{}),
              # routing controller, default: None
                routing_controller=(ContinuousRouter, {}),
              # initial speed, default: 0
                initial_speed=0,
              # number of vehicles, default: 1 
                num_vehicles=2,
                
                car_following_params=cf_parameter
              # speed mode, default: "right_of_way"
                #speed_mode="aggressive",
              # lane change mode, default: "no_lat_collide"
                #lane_change_mode="aggressive", 
              # car following parameter, default: None
                #sumo_car_following_params=cf_parameter,
              # lane change parameter, default: None
                #sumo_lc_params=lc_parameter
)

### Flow Parameter

In [8]:
# Creating flow_params. Make sure the dictionary keys are as specified. 
flow_params = dict( # name of the experiment
                      exp_tag=experiment_name,
                    # name of the flow environment the experiment is running on
                      env_name=env_name,
                    # name of the scenario class the experiment uses
                      scenario=scenario_name,
                    # simulator that is used by the experiment
                      simulator='traci',
                    # sumo-related parameters (see flow.core.params.SumoParams)
                      sim=sumo_params,
                    # environment related parameters (see flow.core.params.EnvParams)
                      env=env_params,
                    # network-related parameters (see flow.core.params.NetParams and
                    # the scenario's documentation or ADDITIONAL_NET_PARAMS component)
                      net=net_params,
                    # vehicles to be placed in the network at the start of a rollout 
                    # (see flow.core.vehicles.Vehicles)
                      veh=vehicles,
                   # (optional) parameters affecting the positioning of vehicles upon 
                   # initialization/reset (see flow.core.params.InitialConfig)
                      initial=initial_config
                )

# B) Training

In [9]:
import json

import ray
try:
    from ray.rllib.agents.agent import get_agent_class
except ImportError:
    from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env

from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder

In [10]:
# number of parallel workers
N_CPUS = 2
# number of rollouts per training iteration
N_ROLLOUTS = 20

ray.init(redirect_output=True, num_cpus=N_CPUS+1)

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-02-19_23-46-01_11033/logs.
Waiting for redis server at 127.0.0.1:15920 to respond...
Waiting for redis server at 127.0.0.1:47122 to respond...
Starting the Plasma object store with 6.554658406 GB memory using /dev/shm.

View the web UI at http://localhost:8889/notebooks/ray_ui.ipynb?token=7e65830e1a33e479a0c15cfe28be83250a74accd79dd1994



{'node_ip_address': '192.168.2.102',
 'object_store_addresses': ['/tmp/ray/session_2019-02-19_23-46-01_11033/sockets/plasma_store'],
 'raylet_socket_names': ['/tmp/ray/session_2019-02-19_23-46-01_11033/sockets/raylet'],
 'redis_address': '192.168.2.102:15920',
 'webui_url': 'http://localhost:8889/notebooks/ray_ui.ipynb?token=7e65830e1a33e479a0c15cfe28be83250a74accd79dd1994'}

In [11]:
# The algorithm or model to train. This may refer to "
#      "the name of a built-on algorithm (e.g. RLLib's DQN "
#      "or PPO), or a user-defined trainable function or "
#      "class registered in the tune registry.")
alg_run = "PPO"

agent_cls = get_agent_class(alg_run)
config = agent_cls._default_config.copy()
config["num_workers"] = N_CPUS  # number of parallel workers
config["train_batch_size"] = HORIZON * N_ROLLOUTS  # batch size
config["gamma"] = 0.999  # discount rate
config["model"].update({"fcnet_hiddens": [64, 32]})  # size of hidden layers in network
config["use_gae"] = True  # using generalized advantage estimation
config["lambda"] = 0.97  
#config["sgd_minibatch_size"] = min(16 * 1024, config["train_batch_size"])  # stochastic gradient descent
#config["sample_batch_size"] = config["train_batch_size"]/config["num_workers"] # 200 default, trotzdem zu hoch?
config["kl_target"] = 0.02  # target KL divergence
config["num_sgd_iter"] = 10  # number of SGD iterations
config["horizon"] = HORIZON  # rollout horizon

# save the flow params for replay
flow_json = json.dumps(flow_params, cls=FlowParamsEncoder, sort_keys=True,
                       indent=4)  # generating a string version of flow_params
config['env_config']['flow_params'] = flow_json  # adding the flow_params to config dict
config['env_config']['run'] = alg_run

# Call the utility function make_create_env to be able to 
# register the Flow env for this experiment
create_env, gym_name = make_create_env(params=flow_params, version=0)

# Register as rllib env with Gym
register_env(gym_name, create_env)

In [12]:
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,  # RL algorithm to run
        "env": gym_name,  # environment name generated earlier
        "config": {  # configuration params (must match "run" value)
            **config
        },
        "checkpoint_freq": 1,  # number of iterations between checkpoints
        "max_failures": 999,
        "stop": {  # stopping conditions
            "training_iteration": 1000,  # number of iterations to stop after
        },
    },
})

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/3 CPUs, 0/1 GPUs
Memory usage on this node: 3.7/16.4 GB

Created LogSyncer for /home/thorsten/ray_results/IntersectionExample/PPO_IntersectionEnv-v0_0_2019-02-19_23-46-024o7i5y8k -> 
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 3.8/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_IntersectionEnv-v0_0:	RUNNING

Result for PPO_IntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-19_23-47-32
  done: false
  episode_len_mean: 450.14285714285717
  episode_reward_max: 148.8779501735844
  episode_reward_mean: 44.48329615873053
  episode_reward_min: -61.94444688976626
  episodes_this_iter: 21
  episodes_total: 21
  experiment_id: 9d2cfd4ba2f7483a838320c419e6817d
  hostname: Gandalf
  info:
    default:
      cur_kl_coeff: 0.20000000298023224
      cur_lr: 4.999999873689376e-05
      entropy: 2.833007335662842


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 4.3/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_IntersectionEnv-v0_0:	RUNNING [pid=11085], 130 s, 6 iter, 60000 ts, 89.9 rew

Result for PPO_IntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-19_23-49-25
  done: false
  episode_len_mean: 311.34
  episode_reward_max: 231.8733811690149
  episode_reward_mean: 97.41129425941595
  episode_reward_min: -60.91913766565617
  episodes_this_iter: 35
  episodes_total: 191
  experiment_id: 9d2cfd4ba2f7483a838320c419e6817d
  hostname: Gandalf
  info:
    default:
      cur_kl_coeff: 0.0031250000465661287
      cur_lr: 4.999999873689376e-05
      entropy: 2.8212931156158447
      kl: 0.009107247926294804
      policy_loss: -0.004174435045570135
      total_loss: 337.58843994140625
      vf_explained_var: 0.2222345769405365
      vf_loss: 337.59259033203125
    grad_time_ms: 1761.715


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 4.3/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_IntersectionEnv-v0_0:	RUNNING [pid=11085], 235 s, 12 iter, 120000 ts, 127 rew

Result for PPO_IntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-19_23-51-12
  done: false
  episode_len_mean: 190.04
  episode_reward_max: 208.45330724732423
  episode_reward_mean: 106.70133355772091
  episode_reward_min: -58.76608713157784
  episodes_this_iter: 50
  episodes_total: 499
  experiment_id: 9d2cfd4ba2f7483a838320c419e6817d
  hostname: Gandalf
  info:
    default:
      cur_kl_coeff: 4.882812572759576e-05
      cur_lr: 4.999999873689376e-05
      entropy: 2.8064470291137695
      kl: 0.005517065990716219
      policy_loss: -0.003812932875007391
      total_loss: 553.7919921875
      vf_explained_var: 0.32349953055381775
      vf_loss: 553.7957763671875
    grad_time_ms: 1613.379
 

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 4.4/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_IntersectionEnv-v0_0:	RUNNING [pid=11085], 341 s, 18 iter, 180000 ts, 125 rew

Result for PPO_IntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-19_23-52-57
  done: false
  episode_len_mean: 179.29
  episode_reward_max: 170.6275329234709
  episode_reward_mean: 133.74612213180018
  episode_reward_min: -61.834554337361666
  episodes_this_iter: 57
  episodes_total: 815
  experiment_id: 9d2cfd4ba2f7483a838320c419e6817d
  hostname: Gandalf
  info:
    default:
      cur_kl_coeff: 3.051757857974735e-06
      cur_lr: 4.999999873689376e-05
      entropy: 2.6840670108795166
      kl: 0.008621570654213428
      policy_loss: -0.003413783386349678
      total_loss: 393.6387634277344
      vf_explained_var: 0.48050636053085327
      vf_loss: 393.6422119140625
    grad_time_ms: 1616.13

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 4.3/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_IntersectionEnv-v0_0:	RUNNING [pid=11085], 447 s, 24 iter, 240000 ts, 124 rew

Result for PPO_IntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-19_23-54-44
  done: false
  episode_len_mean: 146.87
  episode_reward_max: 185.19235873923145
  episode_reward_mean: 131.20924362184655
  episode_reward_min: -63.58489935308091
  episodes_this_iter: 66
  episodes_total: 1209
  experiment_id: 9d2cfd4ba2f7483a838320c419e6817d
  hostname: Gandalf
  info:
    default:
      cur_kl_coeff: 9.536743306171047e-08
      cur_lr: 4.999999873689376e-05
      entropy: 2.6573469638824463
      kl: 0.011125963181257248
      policy_loss: -0.003384200157597661
      total_loss: 609.048095703125
      vf_explained_var: 0.39044174551963806
      vf_loss: 609.0514526367188
    grad_time_ms: 1618.5


...

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 6.5/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_IntersectionEnv-v0_0:	RUNNING [pid=11085], 17112 s, 972 iter, 9720000 ts, 184 rew

Result for PPO_IntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-20_04-33-03
  done: false
  episode_len_mean: 137.06
  episode_reward_max: 224.43945981462542
  episode_reward_mean: 184.64403816480507
  episode_reward_min: 95.6359946443007
  episodes_this_iter: 74
  episodes_total: 73423
  experiment_id: 9d2cfd4ba2f7483a838320c419e6817d
  hostname: Gandalf
  info:
    default:
      cur_kl_coeff: 7.822571572683087e-19
      cur_lr: 4.999999873689376e-05
      entropy: 0.7021815180778503
      kl: 0.031148601323366165
      policy_loss: 0.003287645522505045
      total_loss: 5.456977844238281
      vf_explained_var: 0.998306155204773
      vf_loss: 5.453690052032471
    grad_time_ms: 1560.6

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 6.5/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_IntersectionEnv-v0_0:	RUNNING [pid=11085], 17215 s, 978 iter, 9780000 ts, 189 rew

Result for PPO_IntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-20_04-34-46
  done: false
  episode_len_mean: 138.57
  episode_reward_max: 224.44246293353558
  episode_reward_mean: 182.34419397365855
  episode_reward_min: 67.99244964963736
  episodes_this_iter: 73
  episodes_total: 73860
  experiment_id: 9d2cfd4ba2f7483a838320c419e6817d
  hostname: Gandalf
  info:
    default:
      cur_kl_coeff: 7.822571572683087e-19
      cur_lr: 4.999999873689376e-05
      entropy: 0.8439425826072693
      kl: 0.012609871104359627
      policy_loss: -0.0018842276185750961
      total_loss: 37.14928436279297
      vf_explained_var: 0.989680826663971
      vf_loss: 37.15116882324219
    grad_time_ms: 155

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 6.6/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_IntersectionEnv-v0_0:	RUNNING [pid=11085], 17320 s, 984 iter, 9840000 ts, 188 rew

Result for PPO_IntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-20_04-36-32
  done: false
  episode_len_mean: 139.14
  episode_reward_max: 224.88901873799685
  episode_reward_mean: 188.03874313036198
  episode_reward_min: 100.33296690379643
  episodes_this_iter: 72
  episodes_total: 74290
  experiment_id: 9d2cfd4ba2f7483a838320c419e6817d
  hostname: Gandalf
  info:
    default:
      cur_kl_coeff: 7.822571572683087e-19
      cur_lr: 4.999999873689376e-05
      entropy: 0.7464337348937988
      kl: 0.023264002054929733
      policy_loss: -0.0005469695897772908
      total_loss: 17.144025802612305
      vf_explained_var: 0.9949612617492676
      vf_loss: 17.144569396972656
    grad_time_ms:

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 6.6/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_IntersectionEnv-v0_0:	RUNNING [pid=11085], 17424 s, 990 iter, 9900000 ts, 185 rew

Result for PPO_IntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-20_04-38-16
  done: false
  episode_len_mean: 140.05
  episode_reward_max: 223.88216065726056
  episode_reward_mean: 185.59784112372373
  episode_reward_min: 67.10253205408563
  episodes_this_iter: 70
  episodes_total: 74721
  experiment_id: 9d2cfd4ba2f7483a838320c419e6817d
  hostname: Gandalf
  info:
    default:
      cur_kl_coeff: 7.822571572683087e-19
      cur_lr: 4.999999873689376e-05
      entropy: 0.9590182304382324
      kl: 0.01851075328886509
      policy_loss: -0.0018294703913852572
      total_loss: 43.55040740966797
      vf_explained_var: 0.9872342348098755
      vf_loss: 43.55223846435547
    grad_time_ms: 155

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 6.5/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_IntersectionEnv-v0_0:	RUNNING [pid=11085], 17529 s, 996 iter, 9960000 ts, 186 rew

Result for PPO_IntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-02-20_04-40-01
  done: false
  episode_len_mean: 136.5
  episode_reward_max: 217.1559557459586
  episode_reward_mean: 180.06198064715102
  episode_reward_min: 104.74594083140695
  episodes_this_iter: 74
  episodes_total: 75155
  experiment_id: 9d2cfd4ba2f7483a838320c419e6817d
  hostname: Gandalf
  info:
    default:
      cur_kl_coeff: 1.173386097793981e-18
      cur_lr: 4.999999873689376e-05
      entropy: 0.5226399898529053
      kl: 0.033098723739385605
      policy_loss: 0.0003526665095705539
      total_loss: 12.71113395690918
      vf_explained_var: 0.9959396123886108
      vf_loss: 12.710780143737793
    grad_time_ms: 156